## Reinforcement Learning Environment

In [47]:
import gym
from gym import spaces
import pygame
import numpy as np
from typing import Dict, Tuple, List
import csv

In [48]:
class Transition:
    state: np.array
    action: str 
    next_state: np.array
    reward: float 

class Env: 
    _condition_symptom_probabilities: Dict[str, List[Tuple[str, float]]] # conditions with symptoms and their probabilities
    _actions: set[str] # symptoms
    _init_state: np.array
    _current_state: np.array
    _img: np.array
    _condition: str
    _symptoms_of_condition: Dict[str, float] # symptoms of conditions
    
    def __init__(self,
                 img: np.array,
                 condition: str,
                ) -> None:  
        self._img = img
        self._condition = condition

        # init condition_symptom_probabilities from health knowledge graph
        self._condition_symptom_probabilities= dict()
        with open('HealthKnowledgeGraph.csv', newline='') as csvfile:
            reader = csv.reader(csvfile, delimiter=',')
            reader.__next__() # skip header
            for condition in reader:
                self._condition_symptom_probabilities[condition[0]] = list()
                for symptom_prob in condition[1].split(','):
                    # examples for symptom_prob: pain (0.318), fever (0.119) or swelling (0.112)
                    symptom = symptom_prob.split('(')[0].strip()
                    prob = float(symptom_prob.split('(')[1].split(')')[0])
                    self._condition_symptom_probabilities[condition[0]].append((symptom, prob))

        # init symptoms_of_condition for easier access
        self._symptoms_of_condition = dict()
        for symptom in self._condition_symptom_probabilities[self._condition]:
            self._symptoms_of_condition[symptom[0]] = symptom[1] 
    
        # init actions
        self._actions = set()
        for condition in self._condition_symptom_probabilities.keys():
            for symptom in self._condition_symptom_probabilities[condition]:
                self._actions.add(symptom[0])         

        # init init_state = vector with cnn output (probabilities per condition) and history of asked symptoms (0=not asked, 1=symptom is present, -1=symptom is not present)
        visual_prior = np.random.uniform(size=(len(self._condition_symptom_probabilities.keys()))) #TODO: replace with cnn output
        self._init_state = np.concatenate((visual_prior,np.zeros((len(self._actions)))), axis=0)
        self._current_state = self._init_state
        #TODO 

    def has_symptom(self, symptom: str) -> bool:
        if symptom not in self._symptoms_of_condition:
            return False
        else:
            phi = np.random.uniform()
            return phi <= self._symptoms_of_condition[symptom]  #TODO: mit excel überprüfen ob sinn macht?

    def step(self, action: str) -> Transition:
        #TODO
        pass
    
    def reset(self) -> None:
        #TODO
        pass

### Test cases

In [54]:
#Testing simulated patient answers
myEnv=Env(np.array([]), 'abscess')
print("Symptoms for abscess:")
print(myEnv._condition_symptom_probabilities['abscess'])
n=0
prob=0
for i in range(10000):
    n+=1
    if myEnv.has_symptom('lump'):
        prob+=1 
print("\nProbability of lump: " + str(prob/n))

Symptoms for abscess:
[('pain', 0.318), ('fever', 0.119), ('swelling', 0.112), ('redness', 0.094), ('chills', 0.092), ('infection', 0.083), ('cyst', 0.047), ('tenderness', 0.037), ('rectal pain', 0.026), ('lesion', 0.025), ('lump', 0.023), ('sore throat', 0.021), ('facial swelling', 0.016), ('pimple', 0.016), ('discomfort', 0.014), ('difficulty swallowing', 0.013), ('cavity', 0.013), ('night sweats', 0.007), ('severe pain', 0.007), ('abdominal pain', 0.007), ('painful swallowing', 0.007), ('back pain', 0.006)]

Probability of lump: 0.0237
